### <b>Image augmentation for INPUT: <b style="color:red">128x128</b></b>

In [3]:
import os
import cv2
import numpy as np
import shutil

In [54]:
class Augmentation:
    def __init__(self, random_bound=0.5):
        self.random_bound = random_bound
    
    def augment_image(self, image, rotate=True):
        # Randomly flip the image horizontally (left-right)
        if np.random.rand() > self.random_bound:
            image = cv2.flip(image, 1)

        # Fixed rotations (90°, 45°, 180°)
        if rotate:
            # Choose a random fixed angle from the list
            angle = np.random.choice([90, 180, 270])
            height, width = image.shape[:2]
            center = (width // 2, height // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
            image = cv2.warpAffine(image, rotation_matrix, (width, height))

        # Random brightness adjustment
        factor = np.random.uniform(1.0, 1.05)
        image = cv2.convertScaleAbs(image, alpha=factor, beta=0)
        return image
    
    def augment_image_file(self, path_to_input_image, path_to_output_image, rotate=True):
        image = cv2.imread(path_to_input_image)
        image = self.augment_image(image)
        cv2.imwrite(path_to_output_image, image)

#### HELPING ####

In [65]:
n_images = 100
augmenter = Augmentation()
path = "./final-dataset/imbalanced_flags/I"
category = "I"
files = os.listdir(path)
last_id = -1
for file in files:
    print(file)
    id = int(file.split("_")[1].split(".")[0])
    if id > last_id:
        last_id = id
for i in range(n_images):
    selected_id = np.random.randint(0, last_id-1) + 1
    if selected_id < 10:
        selected_id = f"0{selected_id}"
    last_id = last_id + 1
    path_to_selected_image = f"{path}/{category}_{selected_id}.jpg"
    path_to_generate_image = f"{path}/{category}_{last_id}.jpg"
    print(f"Path to selected image: {path_to_selected_image}")
    print(f"Path to generate image: {path_to_generate_image}\n")
    augmenter.augment_image_file(
        path_to_selected_image,
        path_to_generate_image,
        True
    )

I_01.jpg
I_02.jpg
I_03.jpg
I_04.jpg
I_05.jpg
I_06.jpg
I_07.jpg
I_08.jpg
I_09.jpg
I_10.jpg
I_100.jpg
I_101.jpg
I_102.jpg
I_103.jpg
I_104.jpg
I_105.jpg
I_106.jpg
I_107.jpg
I_108.jpg
I_109.jpg
I_11.jpg
I_110.jpg
I_111.jpg
I_112.jpg
I_113.jpg
I_114.jpg
I_115.jpg
I_116.jpg
I_117.jpg
I_12.jpg
I_13.jpg
I_14.jpg
I_15.jpg
I_16.jpg
I_17.jpg
I_18.jpg
I_19.jpg
I_20.jpg
I_21.jpg
I_22.jpg
I_23.jpg
I_24.jpg
I_25.jpg
I_26.jpg
I_27.jpg
I_28.jpg
I_29.jpg
I_30.jpg
I_31.jpg
I_32.jpg
I_33.jpg
I_34.jpg
I_35.jpg
I_36.jpg
I_37.jpg
I_38.jpg
I_39.jpg
I_40.jpg
I_41.jpg
I_42.jpg
I_43.jpg
I_44.jpg
I_45.jpg
I_46.jpg
I_47.jpg
I_48.jpg
I_49.jpg
I_50.jpg
I_51.jpg
I_52.jpg
I_53.jpg
I_54.jpg
I_55.jpg
I_56.jpg
I_57.jpg
I_58.jpg
I_59.jpg
I_60.jpg
I_61.jpg
I_62.jpg
I_63.jpg
I_64.jpg
I_65.jpg
I_66.jpg
I_67.jpg
I_68.jpg
I_69.jpg
I_70.jpg
I_71.jpg
I_72.jpg
I_73.jpg
I_74.jpg
I_75.jpg
I_76.jpg
I_77.jpg
I_78.jpg
I_79.jpg
I_80.jpg
I_81.jpg
I_82.jpg
I_83.jpg
I_84.jpg
I_85.jpg
I_86.jpg
I_87.jpg
I_88.jpg
I_89.jpg
I_90.jpg
I_91.jpg
I

#### -------------------- ####

In [245]:
class AugmentationBalancer:
    def __init__(self, random_bound=0.5):
        self.augmentation = Augmentation(random_bound=random_bound)
    
    def fit(self, path_to_folder, delta=5, debug=False):
        self.path_to_folder=path_to_folder        
        directories = os.listdir(path_to_folder)
        real_directories = {}
        self.to_generate = {}
        for directory in directories:
            if os.path.isdir(f"{path_to_folder}/{directory}"):
                count = len(os.listdir(f"{path_to_folder}/{directory}"))
                real_directories[directory]=count
        max_id = max(real_directories)
        self.total_classes = [key for key in real_directories]
        for directory in real_directories:
            diff = real_directories[max_id] - real_directories[directory]
            if(diff>delta):
                self.to_generate[directory]=diff
        if debug:
            print(self.to_generate)
        
    def balance(self, path_to_folder, debug=False):
        for category in self.total_classes:
            source_folder = f"{self.path_to_folder}/{category}"
            destination_folder =f"{path_to_folder}/{category}"
            if os.path.exists(destination_folder) and os.path.isdir(destination_folder):
                shutil.rmtree(destination_folder)
            shutil.copytree(source_folder, destination_folder)
        if debug:
            print(self.to_generate)
        for category in self.to_generate:
            last_id = -1
            for file in os.listdir(f"{path_to_folder}/{category}"):
                id = int(file.split("_")[1].split(".")[0])
                if id > last_id:
                    last_id = id
            for i in range(self.to_generate[category]):
                selected_id = np.random.randint(0, last_id-1) + 1
                if selected_id < 10:
                    selected_id = f"0{selected_id}"
                last_id = last_id + 1
                path_to_selected_image = f"{path_to_folder}/{category}/{category}_{selected_id}.jpg"
                path_to_generate_image = f"{path_to_folder}/{category}/{category}_{last_id}.jpg"
                if debug:
                    print(f"Path to selected image: {path_to_selected_image}")
                    print(f"Path to generate image: {path_to_generate_image}\n")
                self.augmentation.augment_image_file(
                    path_to_selected_image,
                    path_to_generate_image
                )
    
aug_balancer = AugmentationBalancer()
aug_balancer.fit("./dataset/training/imbalanced_flags", debug=True)
aug_balancer.balance("./dataset/training/balanced_flags/", debug=True)

{'China': 20, 'Germany': 7, 'India': 57}
{'China': 20, 'Germany': 7, 'India': 57}
Path to selected image: ./dataset/training/balanced_flags//China/China_72.jpg
Path to generate image: ./dataset/training/balanced_flags//China/China_80.jpg

Path to selected image: ./dataset/training/balanced_flags//China/China_03.jpg
Path to generate image: ./dataset/training/balanced_flags//China/China_81.jpg

Path to selected image: ./dataset/training/balanced_flags//China/China_73.jpg
Path to generate image: ./dataset/training/balanced_flags//China/China_82.jpg

Path to selected image: ./dataset/training/balanced_flags//China/China_75.jpg
Path to generate image: ./dataset/training/balanced_flags//China/China_83.jpg

Path to selected image: ./dataset/training/balanced_flags//China/China_02.jpg
Path to generate image: ./dataset/training/balanced_flags//China/China_84.jpg

Path to selected image: ./dataset/training/balanced_flags//China/China_05.jpg
Path to generate image: ./dataset/training/balanced_fla